In [2]:
import requests
import pandas as pd

# NYC Open Data API endpoint
base_url = "https://data.cityofnewyork.us/resource/pvqr-7yc4.json"

# Settings
limit = 50000         # Rows per request
offset = 0            # Starting index
max_records = 2000000 # Total you want

# Container for all records
data = []

while offset < max_records:
    params = {
        "$limit": limit,
        "$offset": offset
    }

    response = requests.get(base_url, params=params)

    if response.status_code != 200:
        print(f"Request failed at offset {offset} with status code {response.status_code}")
        break

    batch = response.json()

    if not batch:
        print("No more data returned.")
        break

    data.extend(batch)
    offset += limit

    print(f"✅ Retrieved {len(batch)} records (Total so far: {len(data)})")

# Convert to DataFrame
df = pd.DataFrame(data)

# Save to CSV
df.to_csv("nyc_parking_violations_sample.csv", index=False)
print("📦 Done! Saved 2 million rows to nyc_parking_violations_sample.csv")


✅ Retrieved 50000 records (Total so far: 50000)
✅ Retrieved 50000 records (Total so far: 100000)
✅ Retrieved 50000 records (Total so far: 150000)
✅ Retrieved 50000 records (Total so far: 200000)
✅ Retrieved 50000 records (Total so far: 250000)
✅ Retrieved 50000 records (Total so far: 300000)
✅ Retrieved 50000 records (Total so far: 350000)
✅ Retrieved 50000 records (Total so far: 400000)
✅ Retrieved 50000 records (Total so far: 450000)
✅ Retrieved 50000 records (Total so far: 500000)
✅ Retrieved 50000 records (Total so far: 550000)
✅ Retrieved 50000 records (Total so far: 600000)
✅ Retrieved 50000 records (Total so far: 650000)
✅ Retrieved 50000 records (Total so far: 700000)
✅ Retrieved 50000 records (Total so far: 750000)
✅ Retrieved 50000 records (Total so far: 800000)
✅ Retrieved 50000 records (Total so far: 850000)
✅ Retrieved 50000 records (Total so far: 900000)
✅ Retrieved 50000 records (Total so far: 950000)
✅ Retrieved 50000 records (Total so far: 1000000)
✅ Retrieved 50000 re